# 참조자료

-  https://smartshk.tistory.com/19


-  https://www.baeldung.com/kotlin/threads-coroutines

## 코루틴 모듈 올리기

In [1]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.6.1")

## 모듈 설치 확인하기

In [2]:
:classpath

Current classpath (14 paths):
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/lib-0.11.0-61.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/api-0.11.0-61.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/kotlin-script-runtime-1.7.0-dev-1825.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/kotlin-reflect-1.6.0.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/kotlin-stdlib-1.6.0.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/annotations-13.0.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/kotlin-stdlib-common-1.6.0.jar
/Users/dahlmoon/.m2/repository/org/jetbrains/kotlinx/kotlinx-coroutines-core/1.6.1/kotlinx-coroutines-core-1.6.1.jar
/Users/dahlmoon/.m2/repository/org/jetbrains/kotlinx

## 코루틴 모듈확인하기

In [3]:
import kotlinx.coroutines.*
import kotlinx.coroutines.channels.*

## 채널은 양방향 처리

- 특정 코루틴에서 송신하고 다른 코루틴에서 수신을 처리한다

In [4]:
fun main() {
    val channel = Channel<Int>() 
    println(channel.javaClass)

    CoroutineScope(Dispatchers.Default).launch { 
        channel.send(100)
    }
    
    runBlocking {

        delay(1000)
        println(channel.receive())
        channel.close()
    }
}

main()



class kotlinx.coroutines.channels.RendezvousChannel
100


In [5]:
fun main() {

    val channel = Channel<Int>() 
    CoroutineScope(Dispatchers.Default).launch { 
        repeat(10) { channel.send(it) } 
    } 

    CoroutineScope(Dispatchers.Default).launch {
        val num = channel.receive()                  // 1 
        println("한개 처리 : $num ")
        var count =0 
        for(x in channel) {                         // 2 
            println("for 순환 : $x ") 
            count++
            if (count > 4) delay(1000)
        } 
    }
    
    CoroutineScope(Dispatchers.Default).launch {
  
        channel.consumeEach {                       // 3 
            println("consume 순환 : $it") } 
    }

}

main()

한개 처리 : 0 
for 순환 : 1 
for 순환 : 2 
for 순환 : 4 
consume 순환 : 3
for 순환 : 5 
consume 순환 : 6
for 순환 : 7 
consume 순환 : 8
consume 순환 : 9


## Channel

개념상 BlockingQueue와 매우 비슷하다. 차이점이 있다면 BlockingQueue는 값을 전달하는 과정에서 스레드를 block하지만, 

Channel은 스레드를 block하지 않고 suspending 한다.

In [6]:
// interface Channel<E> : SendChannel<E> , ReceiveChannel<E>  // 채널은 송신과 수신 채널을 묶는 개념 
// fun <E> Channel(                                           // 채널함수 
//    capacity: Int = RENDEZVOUS,                             // 버퍼 용량 
//    onBufferOverflow: BufferOverflow = BufferOverflow.SUSPEND, 
//    onUndeliveredElement: (E) -> Unit? = null
// ): Channel<E>

fun main() = runBlocking {
        val chan = Channel<Int>()
        launch {
            for (x in 1..5) chan.send(x *x)                   // 값을 보냄
        }
        launch {
            repeat(3) { 
                delay(10)
                println("코루틴1 : " + chan.receive())          // 값을 받음 
            }                                                  // 
        }
        launch {
            repeat(2) { 
                delay(20)
                println("코루틴2 : " +chan.receive())           // 값을 받음 
            }                                                 // 
        }
        delay(200)
        chan.close()                                      // 더 보낼 값이 없음, close
        println("Done!")
}
main()

코루틴1 : 1
코루틴2 : 4
코루틴1 : 9
코루틴1 : 16
코루틴2 : 25
Done!


## 문자열 채널 전달 

In [7]:
fun from_ont_to_another() {
    runBlocking {

        val channel = Channel<String>()                       // 문자열 채널 생성 

        launch {                                              // 전송 코루틴 
            channel.send("코틀린 세상에 들어온 것을 환영!")
            channel.close()
        }
        val result = async {                                  // 수신 코루틴 
            channel.receive()
        }

        println(result.await())                               // 코루틴 반환값 출력
        println(result.await() == "코틀린 세상에 들어온 것을 환영!") // 전달된 값 비교
    }
}

from_ont_to_another()

코틀린 세상에 들어온 것을 환영!
true


## 랑데뷰 채널 연속 전송

In [8]:
fun Rendezvous_Channel() {
    runBlocking {
        val basket = Channel<String>()
        launch {                                              // 코루틴에서 순환문으로 전송
            val fruits = listOf("사과", "오렌지")
            for (fruit in fruits) {
                println("송신 1 코루틴 : $fruit")
                 delay(100)
                basket.send(fruit)
            }
            for (fruit in fruits) {                            // 코루틴에서 순환문으로 전송
                println("송신 2 코루틴 : $fruit")
                delay(100)
                basket.send(fruit)
            } 
            basket.close()                                      // 채널 닫음 
        }
        launch(Dispatchers.IO) {                                // 4개의 코뤁틴을 만듬 
            repeat(4) {                                         // 각각 출력 
                delay(100)
                println(Thread.currentThread().name)
                println("수신 코루틴 : ${basket.receive()}")
            }
        }
    }
}

Rendezvous_Channel()

송신 1 코루틴 : 사과
DefaultDispatcher-worker-1
수신 코루틴 : 사과
송신 1 코루틴 : 오렌지
DefaultDispatcher-worker-1
수신 코루틴 : 오렌지
송신 2 코루틴 : 사과
DefaultDispatcher-worker-1
송신 2 코루틴 : 오렌지
수신 코루틴 : 사과
DefaultDispatcher-worker-1
수신 코루틴 : 오렌지


## 버퍼 채널 처리 

In [9]:
fun buffer_Channel() {  
    runBlocking {
        val basket = Channel<String>(2)                      // 채널 
        val fruits = listOf("바나나", "딸기", "앵두")
        launch {                                             // 송신 고루틴 1
            for (fruit in fruits) {
                println(" 송신 1: $fruit")
                basket.send(fruit)
            }
        }
        launch {                                             // 송신 코루틴 2
            for (fruit in fruits) {
                println(" 송신 2: $fruit")
                basket.send(fruit)
            }
        }
        launch {                                             // 수신 코루틴 
            repeat(6) {
                delay(100)
                println(" 수신 : ${basket.receive()}")
            }
        }  
        delay(3000)
        basket.close()                                        // 채널 닫기 
    }
}
buffer_Channel()

 송신 1: 바나나
 송신 1: 딸기
 송신 1: 앵두
 송신 2: 바나나
 수신 : 바나나
 수신 : 딸기
 송신 2: 딸기
 수신 : 앵두
 송신 2: 앵두
 수신 : 바나나
 수신 : 딸기
 수신 : 앵두


## 무한버퍼 처리 

In [10]:
fun unlimited_Channel() {
    runBlocking {
        val channel = Channel<Int>(Channel.UNLIMITED)    // 버퍼 제한을 안줌 
        launch {                                         // 송신 코루틴
            repeat(3) {
                delay(30)
                println(" 송신 1 :  $it")                 // 인덱스 값을 전송
                channel.send(it)
            }
        }
        launch {                                         // 송신 코루틴 
            repeat(3) {
                delay(50)
                println(" 송신 2 :  $it")
                channel.send(it)
            }
        }
        launch {                                          // 수신 코루틴 
            repeat(6) {
                delay(100)
                println(" 수신 :  ${channel.receive()}")
            }
        }
       delay(2000)
       channel.close()                                     // 채널 닫기
    }
}

unlimited_Channel()

 송신 1 :  0
 송신 2 :  0
 송신 1 :  1
 송신 1 :  2
 수신 :  0
 송신 2 :  1
 송신 2 :  2
 수신 :  0
 수신 :  1
 수신 :  2
 수신 :  1
 수신 :  2


## 중첩쓰기 버퍼 처리 

In [11]:
fun conflated_channel() {
    runBlocking {
        val basket = Channel<String>(Channel.CONFLATED)         // 사용하지 않은 버퍼는 중첩시킴 
        val animal = listOf("강아지", "고양이", "거북이")
        launch {                                                // 코루틴에서 채널로 송신 
            for (pet in animal) {
                delay(10)
                println(" 송신 1 : $pet")
                basket.send(pet)
            }
        }
    
        launch {                                                // 수신
                delay(300)
                println(" 수신 :  ${basket.receive()}")
        }
    }
}

conflated_channel()

 송신 1 : 강아지
 송신 1 : 고양이
 송신 1 : 거북이
 수신 :  거북이


## Producer-Consumer 패턴


- produce 코루틴 빌더와 consumeEach 함수를 사용하여 이 패턴을 정확히 구현할 수 있다.

- 두 함수를 사용하면 Channel 객체를 명시적으로 만들지 않아도 된다.

In [12]:
// interface ProducerScope<in E>                          // 생성자스코프 
// : CoroutineScope, SendChannel<E>                       // 코루틴스코프와 센드채널 상속

// fun <E> CoroutineScope.produce(                        // 생산자 빌더 
//    context: CoroutineContext = EmptyCoroutineContext,  // 컨텍스트  
//    capacity: Int = 0,                                  // 용량
//    block: suspend ProducerScope<E>.() -> Unit          // 생산자 스코프의 람다표현식 
// ): ReceiveChannel<E>                                   // 수신 채널 

fun CoroutineScope.produceSquares() = produce {      // 코루틴스코프 내의 확장함수 생성 
    for (x in 1..5) {
        send(x * x)                                  // 값을 하나씩 produce
    }
}

fun main() = runBlocking {
    val squares = produceSquares()                   // 생산자 함수 실행 
    println(squares.javaClass)                       // 수신자 반환
    squares.consumeEach { println(it) }              // 값을 하나씩 consume
    println("Done!")
}

main()

class kotlinx.coroutines.channels.ProducerCoroutine
1
4
9
16
25
Done!


## 파이프라인
파이프라인이란 유한 또는 무한 개의 값을 만드는 코루틴을 의미한다.

In [13]:
fun CoroutineScope.numbers() = produce<Int> {            //  숫자 생성자 
    var x = 1
    while (true) send(x++)                               // 1부터 시작하여 모든 자연수를 produce
}

fun CoroutineScope.square(numbers: ReceiveChannel<Int>): ReceiveChannel<Int> = produce {
    for (x in numbers) send(x * x)                       // 주어진 수의 제곱을 produce
}

fun main() = runBlocking {
    val num = numbers()                                  // 숫자 생성자 코루틴                     
    val squares = square(num)                            // 채널을 전달받은 코루틴 
    repeat(8) {
        println(squares.receive())                       // 처음 8개만 출력
    }
    println("Done!") 
    coroutineContext.cancelChildren()                    // 파이프라인 중단
}

main()

1
4
9
16
25
36
49
64
Done!


## 생산자를 확장함수로 처리

In [14]:
fun CoroutineScope.produceOrders(): ReceiveChannel<String> = produce {
    var x = 1                                                // 생산자를 확장함수로 정의 
    while (true) {
        send("주문번호 . ${x++}")                              // 채널 전송 
        delay(100)
    }
}
fun CoroutineScope.orderProcessor(id: Int, orders: ReceiveChannel<String>) = launch {
    for (order in orders) {                                  // 수신 채널 확장함수 정의
        println("처리번호 : #$id $order")                      //  수신 결과를 처리
    }
}

fun main() = runBlocking {
    val orders = produceOrders()                              // 무한전송 처리 
    repeat(3) {
        orderProcessor(it + 1, orders)                        // 일부만 받아서 처리 
    }
    delay(1000)
    orders.cancel()
}
main()

처리번호 : #1 주문번호 . 1
처리번호 : #1 주문번호 . 2
처리번호 : #2 주문번호 . 3
처리번호 : #3 주문번호 . 4
처리번호 : #1 주문번호 . 5
처리번호 : #2 주문번호 . 6
처리번호 : #3 주문번호 . 7
처리번호 : #1 주문번호 . 8
처리번호 : #2 주문번호 . 9
처리번호 : #3 주문번호 . 10


## 채널을 일시중단함수로 처리

In [15]:
suspend fun fetchWords(channel: SendChannel<String>) {         // 채널을 인자로 받음
    val lists = listOf("코틀린 ", "코루틴 ","동시성")
    for (tweet in lists) {
        delay(100)
        channel.send(tweet)                                    //  채널에 전송 
    }
}

suspend fun fetchVideos(channel: SendChannel<String>) {        // 채널을 인자로 받음 
    val videos = listOf("닥터 후", "스타워즈")
    for (video in videos) {
        delay(100)
        channel.send(video)                                    // 채녈에 전송 
    }
}

fun main() = runBlocking {
    val aggregate = Channel<String>()
    launch { fetchVideos(aggregate) }                          // 전송 채널
    launch { fetchWords(aggregate) }
    launch {                                                   // 수신 채널 
        repeat(5) {
            println(aggregate.receive())
        }
    }
    delay(3000)                                                // 일시중단 
    coroutineContext.cancelChildren()                          // 자식 코루틴 중단 
}

main()

닥터 후
코틀린 
스타워즈
코루틴 
동시성
